In [5]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[5],[11],[15]]

In [7]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float().to('cuda')
        self.y = torch.tensor(y).float().to('cuda')
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

In [8]:
ds = MyDataset(x,y)
dl = DataLoader(ds, batch_size=2, shuffle=True)
for x, y in dl:
    print("x : {},\ny : {}".format(x,y))

x : tensor([[5., 6.],
        [3., 4.]], device='cuda:0'),
y : tensor([[11.],
        [ 5.]], device='cuda:0')
x : tensor([[7., 8.],
        [1., 2.]], device='cuda:0'),
y : tensor([[15.],
        [ 3.]], device='cuda:0')


In [102]:
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    
    def forward(self,x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

In [103]:
mynet = MyNeuralNet().to('cuda')
loss = nn.MSELoss()
from torch.optim import SGD
opt = SGD(mynet.parameters(), lr = 0.001)

In [104]:
import time
loss_history = []
start = time.time()
for _ in range(50):
    for data in dl:
        x,y = data
        opt.zero_grad()
        loss_value = loss(mynet(x),y)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

0.07299637794494629


In [143]:
val_x = [[10,11]]
val_x = torch.tensor(val_x).float().to('cuda')
mynet(val_x)

tensor([[20.1287]], device='cuda:0', grad_fn=<AddmmBackward>)